<h1> <strong> <center> HTP Test Automation </center> </strong> </h1>

In [ ]:
import os
import getpass
import json
import time
import base64
import re
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage
from langchain_core.prompts import PromptTemplate
from docx import Document as DocxDocument
from docx.shared import Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH


In [2]:
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API Key: ")

In [3]:
vision_llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro", temperature=0.0, max_output_tokens=2048)
reasoning_llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro", temperature=0.2)

print("Models initialized successfully.")

Models initialized successfully.


In [4]:
def retry_with_backoff(func, retries=3, initial_delay=10):
    """Retry a function with exponential backoff for rate limit errors."""
    for attempt in range(retries):
        try:
            return func()
        except Exception as e:
            if "RESOURCE_EXHAUSTED" in str(e) or "429" in str(e):
                wait_time = initial_delay * (2 ** attempt)
                print(f"     Rate limited. Waiting {wait_time}s before retry {attempt+1}/{retries}...")
                time.sleep(wait_time)
            else:
                raise
    raise Exception(f"Failed after {retries} retries due to rate limiting.")

def extract_features_from_image(image_path, drawing_type, context_instruction=""):

    with open(image_path, "rb") as image_file:
        image_data = base64.b64encode(image_file.read()).decode("utf-8")

    vision_prompt = f"""
    You are an expert psychometric vision analyzer. 
    {context_instruction}
    
    YOUR TASK: Locate and analyze ONLY the {drawing_type} in the image.
    
    Look specifically for these visual features mentioned in HTP manuals:
    1. Line Quality (Sketchy, heavy, faint, broken)
    2. Size of drawing (Tiny, Big, Average relative to the available space)
    3. Placement on page (Left, Right, Center, Top, Bottom, Edge)
    4. Omissions (Missing hands, feet, eyes, mouth, neck, chimney, branches, roots etc.)
    5. Shading (Face, body, hands, excessive shading)
    6. Distortions or Asymmetry
    7. Specific details (Teeth, long arms, short arms, transparencies, clouds, rain, scars on tree)
    
    Return the output ONLY as a raw JSON object with no markdown formatting.
    Structure:
    {{
      "drawing_type": "{drawing_type}",
      "line_quality": "...",
      "size": "...",
      "placement": "...",
      "omissions": ["list", "of", "missing", "parts"],
      "shading_areas": ["list", "of", "shaded", "areas"],
      "distortions": "...",
      "key_details": ["list", "of", "notable", "features"]
    }}
    """
    
    message = HumanMessage(
        content=[
            {"type": "text", "text": vision_prompt},
            {"type": "image_url", "image_url": f"data:image/jpeg;base64,{image_data}"},
        ]
    )
    
    response = retry_with_backoff(lambda: vision_llm.invoke([message]))
    raw_content = response.content
    
    if isinstance(raw_content, list):
        full_text = ""
        for part in raw_content:
            if isinstance(part, str):
                full_text += part
            elif isinstance(part, dict) and "text" in part:
                full_text += part["text"]
        raw_content = full_text
    
    content = raw_content.strip()
    
    if content.startswith("```json"):
        content = content.replace("```json", "").replace("```", "")
    elif content.startswith("```"):
        content = content.replace("```", "")
    
    try:
        return json.loads(content)
    except json.JSONDecodeError:
        print(f"JSON Parse Error. Raw content was: {content[:100]}...")
        return {
            "drawing_type": drawing_type,
            "error": "Failed to parse JSON from model response"
        }

In [5]:
def analyze_htp_drawings(file_paths):
    
    full_analysis_results = {}
    
    if "combined" in file_paths:
        print("--- Detected Single Sheet Mode (Combined) ---")
        image_path = file_paths["combined"]
        targets = ["House", "Tree", "Person"]
        
        for target in targets:
            print(f"Analyzing {target} from combined sheet...")
            context = f"NOTE: This image contains multiple drawings. IGNORE the others. Focus ONLY on the {target}."
            
            result = extract_features_from_image(image_path, drawing_type=target, context_instruction=context)
            full_analysis_results[target] = result

    else:
        print("--- Detected Separate Sheets Mode ---")
        
        for drawing_type, path in file_paths.items():
            if drawing_type in ["House", "Tree", "Person"]:
                print(f"Analyzing {drawing_type} from {path}...")
                context = "NOTE: This image contains a single drawing on its own sheet."
                
                result = extract_features_from_image(path, drawing_type=drawing_type, context_instruction=context)
                full_analysis_results[drawing_type] = result
                
    return full_analysis_results

In [6]:
# Scenario 1: Use this if you have ONE file with everything
#files_combined = {"combined": r"C:\Users\abdul\Desktop\fyp testing\i(3).jpeg"}
#results = analyze_htp_drawings(files_combined)
#print(json.dumps(results, indent=2))

# Scenario 2: Use this if you have THREE separate files
# files_separate = {
#     "House": "client_house.png", 
#     "Tree": "client_tree.png", 
#     "Person": "client_person.png"
# }
# results = analyze_htp_drawings(files_separate)
# print(json.dumps(results, indent=2))

In [7]:
# --- CELL 2: PERSISTENT SMART KNOWLEDGE BASE (MANUAL + PROCESSED CASES) ---

import json
import os
import time
from langchain_core.documents import Document as LangChainDocument
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS

# --- 1. CONFIGURATION & PATHS ---
BASE_DIR = "htp_recources/" 
DB_FAISS_PATH = "vectorstore/db_faiss"
FORCE_REBUILD = False  # Set to True if you change your PDFs or images

manual_path = os.path.join(BASE_DIR, "book_htp1-82.pdf")
embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")


training_cases = [
    {"id": "Case 1", "images": {"House": "house1.png", "Tree": "tree1.png", "Person": "person1.png"}, "interpretation_pdf": "case1.pdf"},
    {"id": "Case 2", "images": {"House": "house2.png", "Tree": "tree2.png", "Person": "person2.png"}, "interpretation_pdf": "case2.pdf"},
    {"id": "Case 3", "images": {"House": "house3.png", "Tree": "tree3.png", "Person": "person3.png"}, "interpretation_pdf": "case3.pdf"},
    {"id": "Case 4", "images": {"House": "house4.png", "Tree": "tree4.png", "Person": "person4.png"}, "interpretation_pdf": "case4.pdf"}
]

# --- 2. PERSISTENCE LOGIC: LOAD OR BUILD ---

if os.path.exists(DB_FAISS_PATH) and not FORCE_REBUILD:
    print(f"✅ Existing Vector Database found at '{DB_FAISS_PATH}'. Loading...")
    vector_store = FAISS.load_local(
        DB_FAISS_PATH, 
        embeddings, 
        allow_dangerous_deserialization=True
    )
    print("🚀 Knowledge Base loaded from disk. Vision API skipped.")

else:
    print("🔍 No local database found (or Rebuild triggered). Starting full extraction...")
    all_docs = []

    # A. LOAD THE MAIN RULE MANUAL
    if os.path.exists(manual_path):
        print(f" 📖 Loading Rule Manual: {manual_path}...")
        try:
            loader = PyPDFLoader(manual_path)
            manual_pages = loader.load()
            for page in manual_pages:
                page.metadata["category"] = "rule_manual"
                all_docs.append(LangChainDocument(page_content=page.page_content, metadata=page.metadata))
        except Exception as e:
            print(f" ❌ Error loading manual: {e}")
    else:
        print(f"⚠️ Manual file not found at: {manual_path}")

    # B. PROCESS THE 4 TRAINING CASES (Vision + Text)
    if 'analyze_htp_drawings' not in globals():
        print("🛑 CRITICAL ERROR: 'analyze_htp_drawings' function is missing! Run that cell first.")
    else:
        for case in training_cases:
            case_id = case["id"]
            pdf_path = os.path.join(BASE_DIR, case["interpretation_pdf"])
            
            case_images = {key: os.path.join(BASE_DIR, val) for key, val in case["images"].items()}
            
            if not os.path.exists(pdf_path):
                print(f"   ! Skipping {case_id}: PDF missing.")
                continue
                
            print(f"   > Processing {case_id}...")
            
            # Vision Analysis with a tiny sleep to avoid 503 errors
            try:
                print(f"     - Extracting visual features...")
                vision_features = analyze_htp_drawings(case_images)
                vision_text_block = json.dumps(vision_features, indent=2)
                time.sleep(1) # Breath between API calls
            except Exception as e:
                print(f"     ! Vision Error for {case_id}: {e}")
                vision_text_block = "Visual features unavailable."

            # Load Expert PDF
            try:
                loader = PyPDFLoader(pdf_path)
                pdf_pages = loader.load()
                expert_text = "\n".join([p.page_content for p in pdf_pages])
            except Exception as e:
                print(f"     ! PDF Error for {case_id}: {e}")
                continue

            combined_content = f"[SOLVED REFERENCE CASE: {case_id}]\n\nVISUAL FEATURES:\n{vision_text_block}\n\nEXPERT INTERPRETATION:\n{expert_text}"
            
            all_docs.append(LangChainDocument(
                page_content=combined_content,
                metadata={"source": pdf_path, "category": "solved_case", "case_id": case_id}
            ))
            print(f"     + Added {case_id} to collection.")

    # C. CHUNK AND SAVE
    if all_docs:
        print("\n📦 Indexing and saving to disk...")
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        split_docs = text_splitter.split_documents(all_docs)
        
        vector_store = FAISS.from_documents(split_docs, embeddings)
        vector_store.save_local(DB_FAISS_PATH)
        print(f"✅ Database saved to '{DB_FAISS_PATH}'.")
    else:
        print("❌ No documents were processed. Database not created.")

# --- 3. INITIALIZE RETRIEVER ---
if 'vector_store' in locals():
    retriever = vector_store.as_retriever(search_kwargs={"k": 6})
    print("✨ Retriever ready for Reasoning.")

🔍 No local database found (or Rebuild triggered). Starting full extraction...
 📖 Loading Rule Manual: htp_recources/book_htp1-82.pdf...
   > Processing Case 1...
     - Extracting visual features...
--- Detected Separate Sheets Mode ---
Analyzing House from htp_recources/house1.png...
Analyzing Tree from htp_recources/tree1.png...
Analyzing Person from htp_recources/person1.png...
     + Added Case 1 to collection.
   > Processing Case 2...
     - Extracting visual features...
--- Detected Separate Sheets Mode ---
Analyzing House from htp_recources/house2.png...
Analyzing Tree from htp_recources/tree2.png...
Analyzing Person from htp_recources/person2.png...
     + Added Case 2 to collection.
   > Processing Case 3...
     - Extracting visual features...
--- Detected Separate Sheets Mode ---
Analyzing House from htp_recources/house3.png...
Analyzing Tree from htp_recources/tree3.png...
Analyzing Person from htp_recources/person3.png...
     + Added Case 3 to collection.
   > Processing

In [8]:
def generate_interpretation_report(features_json):

    print(f"   > Searching manual for: {features_json['drawing_type']} features...")
    
    omissions = ", ".join(features_json.get('omissions', []))
    details = ", ".join(features_json.get('key_details', []))
    shading = ", ".join(features_json.get('shading_areas', []))
    
    query_specific = (
        f"Interpretation of {features_json['drawing_type']} indicators: "
        f"omission of {omissions}, details like {details}, shading on {shading}"
    )
    
    size = features_json.get('size', 'average')
    lines = features_json.get('line_quality', 'average')
    placement = features_json.get('placement', 'center')
    
    query_general = (
        f"Projective personality interpretation of: "
        f"{size} size drawing, {lines} line quality, {placement} placement on page"
    )
    
    docs_specific = retriever.invoke(query_specific)
    docs_general = retriever.invoke(query_general)
    
    combined_docs_map = {doc.page_content: doc for doc in (docs_specific + docs_general)}
    unique_docs = list(combined_docs_map.values())
    
    context_text = "\n\n".join([doc.page_content for doc in unique_docs])
        
    system_prompt_template = """
    You are an HTP (House-Tree-Person) assessment assistant.
    
    STRICT RULES:
    1. You may ONLY use the provided manual excerpts below as Context.
    2. Do NOT diagnose (e.g., do not say "The patient has depression").
    3. Do NOT claim certainty (Use phrases like "This may suggest...", "Often associated with...").
    4. Use cautious language.
    5. Always mention limitations (reliability/validity).
    
    CRITICAL INTERPRETATION INSTRUCTION:
    The manual often describes rules (like Size, Line Quality, Placement, Shading) in the context of "Human Figures."
    **YOU MUST APPLY THESE GENERAL RULES TO ALL DRAWINGS (HOUSE AND TREE) UNLESS SPECIFICALLY STATED OTHERWISE.**
    Example: If the manual says "Big Figure indicates expansiveness," you should interpret a "Big House" as indicating expansiveness.
    Example: If "Faint lines" mean anxiety for a Person, they mean anxiety for a Tree too.
    
    MANUAL CONTEXT:
    {context}
    
    USER DRAWING FEATURES (JSON):
    {json_data}
    
    TASK:
    1. Create a header: "ANALYSIS OF [DRAWING TYPE]"
    2. Create a "Scoring Rules Identified" section listing the indicators found in the JSON.
    3. Create an "Interpretive Themes" section based on the Manual Context.
       - Apply general rules (Size, Placement, Lines) to this drawing even if the manual mentions "Figure".
    4. List "Follow-Up Questions" specifically for this drawing type.
    5. End with a "Cautions/Limitations" section.
    """
    
    prompt = PromptTemplate(
        input_variables=["context", "json_data"],
        template=system_prompt_template
    )
    
    chain = prompt | reasoning_llm
    response = chain.invoke({"context": context_text, "json_data": json.dumps(features_json)})
    return response.content

In [9]:
# --- DEFINING PATIENT CONTEXT ---
# Define this variable at the top of the cell
PATIENT_CONTEXT_TEXT = """
HISTORY:
This 16-year-old, white female was seen in one of the Colony's Mental Hygiene Clinics. 
Miss A's parents had separated when she was a small child. She lived with a dominant grandmother. 
Miss A. began school at age 7, stopped at age 14 (3rd grade). 
She was placed under care after having sexual relations with various boys. 
Currently tense, mildly depressed; desires to return to grandmother's home. 
Diagnosis: mental deficiency, familial; intellectual status, moron.
"""

# --- MODIFIED SYNTHESIS FUNCTION ---
def generate_synthesis_report(final_reports, patient_history=""):
    print("PHASE 3: CLINICAL SYNTHESIS")
    print("-" * 30)
    print("   > Synthesizing drawings with patient history...")

    house_text = final_reports.get('House', "No House report available.")
    tree_text = final_reports.get('Tree', "No Tree report available.")
    person_text = final_reports.get('Person', "No Person report available.")
    
    combined_data = f"""
    --- HOUSE REPORT --- {house_text}
    --- TREE REPORT --- {tree_text}
    --- PERSON REPORT --- {person_text}
    """

    system_prompt_template = """
    You are a Clinical Psychologist writing a formal HTP Assessment Report.
    
    INPUT DATA:
    1. **Patient History**: {patient_history}
    2. **Drawing Analysis**: {combined_reports}
    
    YOUR TASK:
    Write a Formal HTP Report strictly following the format below.
    **CRITICAL INSTRUCTION**: You MUST Correlate the visual findings with the Patient History.
    - Example: If the history mentions "sexual relations," look for sexual conflict indicators in the Person drawing.
    - Example: If the history mentions "dominant grandmother," look for House/Tree indicators of maternal dependency.
    
    STRICT OUTPUT FORMAT:
    
    QUALITATIVE ANALYSIS
    
    I. Details
    House: [Describe details] ([Interpretation]).
    Tree: [Describe details] ([Interpretation]).
    Person: [Describe details] ([Interpretation]).
    
    II. Proportion
    House: [Describe size/ratio] ([Interpretation]).
    Tree: [Describe size/ratio] ([Interpretation]).
    Person: [Describe size/ratio] ([Interpretation]).
    General: [Overall proportion observation].
    
    III. Perspective
    House: [Placement/Rotation] ([Interpretation]).
    Tree: [Placement/Rotation] ([Interpretation]).
    Person: [Placement/Rotation] ([Interpretation]).
    General: [Overall placement trends].
    
    IV. Line Quality
    House: [Pressure/Stroke] ([Interpretation]).
    Tree: [Pressure/Stroke] ([Interpretation]).
    Person: [Pressure/Stroke] ([Interpretation]).
    General: [Overall energy/control].
    
    IX. Comments (Synthesis of Narrative)
    General: [Synthesize the patient's attitude. Explicitly mention how the drawings reflect the history provided (e.g., "The messy lines correlate with the patient's reported tension...")].
    
    X. Concepts
    [Summarize the core "story" of each drawing. Connect symbols to the patient's life].
    
    SUMMARY
    
    Qualitative Analysis: [List core psychological conflicts found. E.g., "(1) sexual conflict; (2) dependency on mother figure..."].
    
    Impression: [Final Clinical Impression. E.g., "Suggestive of Reactive Depression," "Intellectual Deficiency," etc. Base this on the History + Drawings].
    """
    
    prompt = PromptTemplate(
        input_variables=["combined_reports", "patient_history"],
        template=system_prompt_template
    )
    
    chain = prompt | reasoning_llm
    # We pass the history here
    response = chain.invoke({
        "combined_reports": combined_data, 
        "patient_history": patient_history
    })
    
    print("\n" + "="*40)
    print("FINAL INTEGRATED SYNTHESIS")
    print("="*40)
    print(response.content)
    print("="*40 + "\n")
    
    return response.content

In [10]:
def run_full_htp_assessment(file_paths, patient_history=""):
    
    print("PHASE 1: VISUAL EXTRACTION")
    print("-" * 30)
    all_features = analyze_htp_drawings(file_paths) 
    
    print("\nPHASE 2: CLINICAL INTERPRETATION")
    print("-" * 30)
    final_reports = {}
    
    for drawing_type, features in all_features.items():
        print(f"Generating report for {drawing_type}...")
        try:
            report = generate_interpretation_report(features)
            final_reports[drawing_type] = report
        except Exception as e:
            print(f"Error generating report for {drawing_type}: {e}")
            
    if len(final_reports) > 1:
        # Pass the history to the synthesis engine
        synthesis = generate_synthesis_report(final_reports, patient_history=patient_history)
        final_reports['Synthesis'] = synthesis
    else:
        print("Not enough reports to generate a synthesis.")
            
    return final_reports

In [11]:
# EXECUTION
PATIENT_CONTEXT_TEXT = """
HISTORY:
This 22-year-old, white male was born in Virginia. He is the only child of his father's second marriage with four older half-siblings (youngest is 14 years older). 
Environment was economically and socially above average. Parents are happily adjusted despite a 20-year age gap. 
Mr. G. spent three years in the Army Air Corps and is now attending college. 
Socially, he adjusts easily but attempts to compensate for feelings of inferiority and self-consciousness regarding his height (6'5") by adopting an extraversive mask. 
He exhibits a mild neurotic necessity to structure situations totally. He enjoys being an iconoclast and defending unconventional theories. 
He prefers the concrete/tangible over the abstract. He has egocentricity and a quick temper.
Diagnosis: Normal individual of above average to superior intelligence showing minor reactive patterns due to readjustment to civilian life and forthcoming marriage.
"""

# HISTORY:
# This 16-year-old, white female was seen in one of the Colony's Mental Hygiene Clinics. 
# Miss A's parents had separated when she was a small child. She lived with a dominant grandmother. 
# Miss A. began school at age 7, stopped at age 14 (3rd grade). 
# She was placed under care after having sexual relations with various boys. 
# Currently tense, mildly depressed; desires to return to grandmother's home. 
# Diagnosis: mental deficiency, familial; intellectual status, moron.


# CASE A: If you have separate files
inputs = {
    "House": r"htp_recources/test_house2.png", 
    "Tree":  r"htp_recources/test_tree2.png", 
    "Person":  r"htp_recources/test_person2.png"}
reports = run_full_htp_assessment(inputs, patient_history=PATIENT_CONTEXT_TEXT)

# CASE B: If you have ONE file (Combined Sheet)
#inputs = {"combined": r"C:\Users\abdul\Desktop\fyp testing\i(3).jpeg"}
#reports = run_full_htp_assessment(inputs)

PHASE 1: VISUAL EXTRACTION
------------------------------
--- Detected Separate Sheets Mode ---
Analyzing House from htp_recources/test_house2.png...
Analyzing Tree from htp_recources/test_tree2.png...
Analyzing Person from htp_recources/test_person2.png...

PHASE 2: CLINICAL INTERPRETATION
------------------------------
Generating report for House...
   > Searching manual for: House features...
Generating report for Tree...
   > Searching manual for: Tree features...
Generating report for Person...
   > Searching manual for: Person features...
PHASE 3: CLINICAL SYNTHESIS
------------------------------
   > Synthesizing drawings with patient history...

FINAL INTEGRATED SYNTHESIS
**QUALITATIVE ANALYSIS**

**I. Details**
*   **House:** The drawing includes a path and steps, suggesting a desire for accessibility and contact. However, the omission of a doorknob indicates a countervailing ambivalence about social entry and a potential blockage in interpersonal relationships. The presence o

In [12]:
def save_report_to_word(report_text, filename="HTP_Clinical_Report.docx"):
    doc = DocxDocument()
    
    style = doc.styles['Normal']
    font = style.font
    font.name = 'Calibri'
    font.size = Pt(11)

    title = doc.add_heading('Integrated Clinical Synthesis: HTP Test', 0)
    title.alignment = WD_ALIGN_PARAGRAPH.CENTER
    lines = report_text.split('\n')
    
    for line in lines:
        line = line.strip()
        
        if not line or line == "---" or line == "***" or "=====" in line:
            continue
            
        if re.match(r'^\d+\.\s', line) or line.startswith('###'):
            clean_text = line.replace('###', '').replace('**', '').strip()
            doc.add_heading(clean_text, level=1)
            
        elif line.startswith('**') and line.endswith('**'):
            clean_text = line.replace('**', '').strip()
            doc.add_heading(clean_text, level=2)
            
        elif line.startswith('*') or line.startswith('-'):
            clean_text = line.lstrip('*- ').replace('**', '').strip()
            p = doc.add_paragraph(clean_text, style='List Bullet')
            
        else:
            clean_text = line.replace('**', '')            
            p = doc.add_paragraph(clean_text)
            p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

    doc.save(filename)
    print(f"Successfully created Word Document: {filename}")
    return filename

In [13]:
synthesis_text = reports.get('Synthesis', "No report found.")
file_name = save_report_to_word(synthesis_text, "test_example2.docx")

Successfully created Word Document: test_example2.docx
